In [ ]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.layers import LSTM
from keras.layers import SimpleRNN
from keras.layers import GRU
import tensorflow as tf
import matplotlib.pyplot as plt
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

**Read data**

In [ ]:
df=pd.read_csv("/Users/rahuljauhari/Desktop/research runoff/final destination/Point_all_nasa.csv")

In [ ]:
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)

In [ ]:
monthly_mean = df.resample('M').mean()

In [ ]:
df_actual=pd.read_excel("/Users/rahuljauhari/Desktop/research runoff/Calibrated and Validated.xlsx")

In [ ]:
# select last column
observed_runnoff=df_actual['observed']
observed_runnoff.head()

**Normalization**

In [ ]:
from scipy.stats import zscore
def func(name):
    x=0
    y=0
    inv= 0
    if name=='zscore':
        x_norm = zscore(monthly_mean)
        y_norm = zscore(observed_runnoff)
        x_norm[x_norm > 3] = 2.8
        x_norm[x_norm < -3] = -2.8
        y_norm[y_norm >3] = 2.8
        y_norm[y_norm < -3] = -2.8
        x=x_norm
        y=y_norm
    if name=='StandardScaler':
        scaler = StandardScaler()
        x_scaled = scaler.fit_transform(monthly_mean)
        y_scaled = scaler.fit_transform(observed_runnoff.values.reshape(-1,1))
        x_scaled[x_scaled > 3] = 2.8
        x_scaled[x_scaled < -3] = -2.8
        y_scaled[y_scaled >3] = 2.8
        y_scaled[y_scaled < -3] = -2.8
        x=      x_scaled  
        y=y_scaled
        inv = scaler
        
    if name == 'MinMaxScaler':
        scaler = MinMaxScaler(feature_range=(0,1))
        x_scaled = scaler.fit_transform(monthly_mean)
        y_scaled = scaler.fit_transform(observed_runnoff.values.reshape(-1,1))
        x=      x_scaled  
        y=y_scaled
        inv = scaler
    return x,y,inv

**Train test split**

In [ ]:
mod =SimpleRNN
pre = 'StandardScaler'
act= 'linear'
opt = 'adam'

In [ ]:
# import train test split
from sklearn.model_selection import train_test_split
x,y,inv_scaler= func(pre)
X_train, X_test,y_train,y_test = train_test_split(x,y,test_size=0.3,shuffle=False)

Model

In [ ]:
model1 = Sequential()
model1.add(mod(64,return_sequences=True, input_shape=(X_train.shape[1], 1),activation=act))
# model1.add(Dropout(0.2))      
model1.add(mod(64,activation=act))  
model1.add(Dense(1,activation=act,kernel_regularizer=tf.keras.regularizers.l2(l2=0.01)))
model1.compile(optimizer=opt, loss='mse')

In [ ]:
model1.fit(X_train, y_train, batch_size=10, epochs=100,shuffle=False, use_multiprocessing=True,verbose=0,validation_split=0.1)

In [ ]:
y_train_pred = model1.predict(X_train)
inversed_act_train = inv_scaler.inverse_transform(y_train)
inversed_pred_train = inv_scaler.inverse_transform(y_train_pred)
date= df_actual['Date']
# make a dataframe with date inversed_act_train and inversed_pred_train as columns
df_train = pd.DataFrame({'Date':date.head(inversed_act_train.shape[0]),'Actual':inversed_act_train[:,0],'Predicted':inversed_pred_train[:,0]})
# df_train.head()

In [ ]:
y_test_pred = model1.predict(X_test)
inversed_act_test = inv_scaler.inverse_transform(y_test)
inversed_pred_test = inv_scaler.inverse_transform(y_test_pred)
date= df_actual['Date']
df_test = pd.DataFrame({'Date':date.tail(inversed_act_test.shape[0]),'Actual':inversed_act_test[:,0],'Predicted':inversed_pred_test[:,0]})
# df_test.head()

In [ ]:
# concatenate train and test dataframes
df_final = pd.concat([df_train,df_test],axis=0)
# df_final.to_csv(f"/Users/rahuljauhari/Desktop/research runoff/final destination/{mod}_{pre}_{act}_{opt}_nasa.csv")

**Metric**

In [ ]:
from sklearn.metrics import mean_squared_error
def rmse1(yt, yp): #lower the better
    return np.sqrt(mean_squared_error(yt, yp))
# Kling-Gupta effciency
def kge1(yt, yp): #highqer the better
    r = np.corrcoef(yt, yp,rowvar=False)[0, 1]
    alpha = np.std(yp) / np.std(yt)
    beta = np.mean(yp) / np.mean(yt)
    return 1 - np.sqrt((r - 1)**2 + (alpha - 1)**2 + (beta - 1)**2)
# Normalized standard Error 
def nse1(yt, yp): 
    return 1 - np.sum((yt - yp)**2) / np.sum((yt - np.mean(yt))**2)
    # r squared
def r21(yt, yp): #higher the better
    return 1 - np.sum((yt - yp)**2) / np.sum((yt - np.mean(yt))**2)

**Train**

In [ ]:
yp1 = model1.predict(X_train)
kge = []
r2=[]
rmse =[]
# for i in range(yp1.shape[]):
print("kge:",kge1(y_train, yp1))
print("r2:",r21(y_train, yp1))
print("rmse:",rmse1(y_train, yp1))


**Test**

In [ ]:
yp1 = model1.predict(X_test)
kge = []
r2=[]
rmse =[]
# for i in range(yp1.shape[]):
print("kge:",kge1(y_test, yp1))
print("r2:",r21(y_test, yp1))
print("rmse:",rmse1(y_test, yp1))
